In [3]:
%load_ext autoreload
%autoreload 2
import pprint, json, math, os, sys
# dir_path = "C:\\Users\\rando\\OneDrive\\Documents\\mywork-repo"
# fund_path = r"C:\Users\rando\OneDrive\Documents\PDFDrive\Mar25"
# dry_path = r"C:\Users\rando\OneDrive\Documents\mywork-repo\notebook\DryRun.pdf"

dir_path = "C:\\Users\\Kaustubh.keny\\Projects\\office-work\\mywork-repo"
fund_path = "C:\\Users\\Kaustubh.keny\\Projects\\Mar 25"
dry_path = r"C:\Users\Kaustubh.keny\Projects\office-work\mywork-repo\notebook\DryRun.pdf"
sys.path.append(os.path.abspath(dir_path))
from app.config_loader import load_config_once
conf = load_config_once()


import fitz, pdfplumber, ocrmypdf,camelot
import pandas as pd
import numpy as np
from collections import defaultdict

from app.utils import Helper
from app.parse_regex import *
from app.parse_table import *

dry_path = r'DryRun.pdf'
fin_path = r'\data\input\financial_indices.xlsx'
mutual_fund = Helper.get_fund_paths(fund_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Loaded config with output_folder = None


In [2]:
lines = [
    ((110, 0), (110, 812)),# Vertical line
    ((0, 350), (812, 350)),
    ((570, 0), (570, 812))
]
pages = [12, 14,16]
bboxes = [[0, 120, 180, 812],[180, 85, 360, 812]] #[(0, 85, 180, 812),(180, 85, 360, 812),(0,100,270,812),(0,100,350,812)]
pages = [i for i in range(1,110)]
Helper.draw_lines_on_pdf(sample_path, lines, bboxes, pages, dry_path)

NameError: name 'sample_path' is not defined

In [ ]:
#NIPPON
# nip_jan = r"C:\Users\Kaustubh.keny\Projects\PDF\Jan 25\Nippon India Mutual Fund\33_31-Jan-25_FS.pdf"
# nip_feb = ""
# nip_mar = r"C:\Users\Kaustubh.keny\Projects\PDF\Mar 25\Nippon India Mutual Fund\33_31-Mar-25_FS.pdf"
# nip_apr = r"C:\Users\Kaustubh.keny\Projects\PDF\Apr 25\Nippon India Mutual Fund\33_30-Apr-25_FS.pdf"
# table_parser = TableParser()
# tables = camelot.read_pdf(nip_apr,flavor="stream",pages="129-140")
# dfs = pd.concat([table.df for table in tables], ignore_index=True)

# sr1 = table_parser._get_matching_row_indices(dfs,["Nippon.+?Fund","Scheme\\s*Name"],thresh=2)
# sr2 = table_parser._get_matching_row_indices(dfs,["minimum application"],thresh=1)
# sr2_expanded = {i for idx in sr2 for i in range(idx, idx + 5)}
# sr1_expanded = {i for idx in sr1 for i in range(idx, idx + 1)}

# all_indices = sr1_expanded | sr2_expanded
# valid_indices = sorted(i for i in all_indices if i in dfs.index)

# filtered_df = dfs.loc[valid_indices].reset_index()
# for idx, rows in filtered_df.iterrows():
#     row_val = " ".join([str(i) for i in rows])
#     row_val = SidKimRegex()._normalize_alphanumeric(row_val)
#     # print(row_val)
#     matches = re.findall(r"Nippon.+?(?:Funds?|ETF|Path|Saver|active|financial|allocation|tunities|duration|psu debt|advantage|small cap 250)\s*(?:of Funds?|Fund of Funds?|Funds?|.+?Plan|FoF)?",row_val, re.IGNORECASE)
#     if matches:
#         print(idx, len(matches))
#         print(matches)

In [ ]:
#BAJAJAJ
jan = r"C:\Users\Kaustubh.keny\Projects\PDF\Jan 25\Bajaj finserv Mutual Fund\59_31-Jan-25_FS.pdf"
feb = r"C:\Users\Kaustubh.keny\Projects\PDF\Feb 25\Bajaj finserv Mutual Fund\59_28-Feb-25_FS.pdf"
mar = r"C:\Users\Kaustubh.keny\Projects\PDF\Mar 25\Bajaj finserv Mutual Fund\59_31-Mar-25_FS.pdf"
apr = r"C:\Users\Kaustubh.keny\Projects\PDF\Apr 25\Bajaj finserv Mutual Fund\59_30-Apr-25_FS.pdf"


table_parser = TableParser()
tables = camelot.read_pdf(feb,flavor="lattice",pages="14-17")
dfs = pd.concat([table.df for table in tables], ignore_index=True)
sc1 = table_parser._get_matching_col_indices(dfs,["Bajaj.+?Fund","SCHEME\\s*NAME"],thresh=20)
sc2 = table_parser._get_matching_col_indices(dfs,["Jensen","Standard\\s*Deviation","Information\\s*ratio","Portfolio\\s*Quants","Tracking Error","YTM","Average\\s*Maturity","Sharpe"],thresh=10)
# sc2 = table_parser._get_matching_col_indices(dfs,["Debt\\s*Quant","Modified\\s*Duration","Macaulay","YTM","Average\\s*Maturity","Sharpe"],thresh=20)
all_cols = sorted(set(sc1)) + list(range(sc2[0], dfs.shape[1]))
fdf = dfs.iloc[:, all_cols]
fdf.columns = ["MUTUAL_FUND"] + [f"METRICS_{i}" for i in range(1, fdf.shape[1])]
hdfc_pattern = re.compile(
    r"(Baj.+?(?:FUNDS?|ETF|PATH|INDEX|SAVER)\s*(?:OF FUNDS?|FUND|FUND OF FUNDS|FOF|.+?PLAN|.+?GROWTH)?)",
    re.IGNORECASE
)
fdf.MUTUAL_FUND = table_parser._clean_series(fdf.MUTUAL_FUND,["normalize_alphanumeric"])
fdf.MUTUAL_FUND = fdf.MUTUAL_FUND.apply(lambda x: hdfc_pattern.findall(x)[0] if isinstance(x, str) and hdfc_pattern.findall(x) else "")
fdf = table_parser._clean_dataframe(fdf,["newline_to_space","str_to_pd_NA"])
fdf = fdf.dropna(axis=0, how="all").dropna(axis=1, how="all")
fdf =table_parser._clean_dataframe(fdf,['NA_to_str'])

data = {}
for idx, rows in fdf.iterrows():
    values = list(rows)
    main_scheme_name = str(values[0]).strip() if not pd.isna(values[0]) else ""
    if main_scheme_name:
        temp = main_scheme_name
        if temp not in data:
            data[temp] = {"metrics": []}
        data[temp]["metrics"].append(" ".join(map(str, values[1:])))
    
    if temp:
        data[temp]["metrics"].append(" ".join(map(str, values)))


In [ ]:
#HDFC
table_parser = TableParser()
tables = camelot.read_pdf(hdfc1,flavor="lattice",pages="91-93")
dfs = pd.concat([table.df for table in tables], ignore_index=True)
sc1 = table_parser._get_matching_col_indices(dfs,["HDFC.+?Fund"],thresh=20)
sc2 = table_parser._get_matching_col_indices(dfs,["MINIMUM\\s*APPLICATION\\s*AMOUNT","Additional\\s*Purchase"], thresh=20)

print("Matched columns:", sc1,sc2)
all_cols = list(set(sc1 + sc2))
fdf = dfs.iloc[:, all_cols]
fdf.columns = ["MUTUAL_FUND","MIN_ADD"]
hdfc_pattern = re.compile(
    r"(HDFC.+?(?:FUNDS?|ETF|PATH|INDEX|SAVER)\s*(?:OF FUNDS?|FUND OF FUNDS|FOF|.+?PLAN)?)",
    re.IGNORECASE
)
fdf.MUTUAL_FUND = table_parser._clean_series(fdf.MUTUAL_FUND,["normalize_alphanumeric"])
fdf.MUTUAL_FUND = fdf.MUTUAL_FUND.apply(lambda x: hdfc_pattern.findall(x)[0] if isinstance(x, str) and hdfc_pattern.findall(x) else x)
fdf = table_parser._clean_dataframe(fdf,["newline_to_space","str_to_pd_NA"])
fdf = fdf.dropna(axis=0, how="all").dropna(axis=1, how="all")
fdf =table_parser._clean_dataframe(fdf,['NA_to_str'])

data = {}
for idx, rows in fdf.iterrows():
    values = list(rows)
    main_scheme_name = values[0]
    if main_scheme_name not in data:
        data[main_scheme_name] = {"min_add":values[1]}
    else:
        data[main_scheme_name].update({"min_add_one":values[1]})

In [ ]:
#DSP
jan = r"C:\Users\Kaustubh.keny\Projects\PDF\Jan 25\DSP Mutual Fund\8_31-Jan-25_FS.pdf"
feb = r"C:\Users\Kaustubh.keny\Projects\PDF\Feb 25\DSP Mutual Fund\8_28-Feb-25_FS.pdf"
mar = r"C:\Users\Kaustubh.keny\Projects\PDF\Mar 25\DSP Mutual Fund\8_31-Mar-25_FS.pdf"
apr = r"C:\Users\Kaustubh.keny\Projects\PDF\Apr 25\DSP Mutual Fund\8_30-Apr-25_FS.pdf"

table_parser = TableParser()
tables = camelot.read_pdf(apr,flavor="lattice",pages="107-123")
dfs = pd.concat([table.df for table in tables], ignore_index=True)
sc1 = table_parser._get_matching_col_indices(dfs,["DSP.+?Fund"],thresh=12)
sc2 = table_parser._get_matching_col_indices(dfs,["REGULAR\\s+PLAN","DIRECT\\s+PLAN"], thresh=12)
sc3 = table_parser._get_matching_col_indices(dfs,["Managing this scheme","total work experience"],thresh=12)
print("Matched columns:", sc1,sc2,sc3)
all_cols = list(set(sc1 + sc2 + sc3))
fdf = dfs.iloc[:, all_cols]
fdf["LOAD_STRUCTURE"] = fdf.iloc[:, -1]
fdf.columns = ["MUTUAL_FUND","FUND_MANAGER","MIN_ADD","LOAD_STRUCTURE"]

dsp_pattern = re.compile(
    r"(DSP.+?(?:FUNDS?|ETF|PATH|INDEX|SAVER)\s*(?:OF FUNDS?|FUNDs?|FUND OF FUNDS?|FOF|.+?PLAN)?)",
    re.IGNORECASE
)

fdf.MUTUAL_FUND = table_parser._clean_series(fdf.MUTUAL_FUND,["normalize_alphanumeric"])
fdf.MUTUAL_FUND = fdf.MUTUAL_FUND.apply(lambda x: dsp_pattern.findall(x)[0] if isinstance(x, str) and dsp_pattern.findall(x) else pd.NA)
fdf = table_parser._clean_dataframe(fdf,["newline_to_space","str_to_pd_NA"])
fdf = fdf.dropna(axis=0, how="all").dropna(axis=1, how="all")
fdf =table_parser._clean_dataframe(fdf,['NA_to_str'])

data = {}
for idx, rows in fdf.iterrows():
    values = list(rows)
    main_scheme_name = values[0]
    if main_scheme_name not in data:
        data[main_scheme_name] = {"fund_manager":values[1],"load_structure":values[3],"min_add":values[2]}
    else:
         data[main_scheme_name].update({"fund_manager_one":values[1],"load_structure_one":values[3],"min_add_one":values[2]})
        


In [100]:
print(list(data.keys()))

['dsp flexi cap fund', 'dsp large mid cap fund ', 'dsp large cap fund ', 'dsp mid cap fund', 'dsp india t i g e r fund ', 'dsp small cap fund', 'dsp focused fund ', 'dsp natural resources and new energy fund', 'dsp elss tax saver fund', 'dsp healthcare fund', 'dsp quant fund ', 'dsp value fund', 'dsp banking financial services fund', 'dsp multicap fund', 'dsp business cycle fund', 'dsp 10y g sec fund', 'dsp bond fund', 'dsp banking psu debt fund', 'dsp credit risk fund ', 'dsp gilt fund ', 'dsp savings fund', 'dsp low duration fund', 'dsp short term fund', 'dsp strategic bond fund', 'dsp ultra short fund', 'dsp corporate bond fund', 'dsp floater fund', 'dsp overnight fund', 'dsp liquidity fund', 'dsp arbitrage fund', 'dsp dynamic asset allocation fund ', 'dsp aggressive hybrid fund ', 'dsp equity savings fund', 'dsp regular savings fund', 'dsp multi asset allocation fund', 'dsp nifty 50 equal weight index fund', 'dsp nifty next 50 index fund', 'dsp nifty 50 index fund', 'dsp nifty midc

In [99]:
with open("dsp_apr.json","w") as file:
    json.dump(data,file)

In [ ]:
def get_proper_fund_names(path: str):
    pattern = "(HDFC.*?(?:FUND|Fund|ETF|FO?o?F)\\s*(?:of Funds?|.+?Plan|Fund of Funds?|Fund)?)"
    title = {}   
    with fitz.open(path) as doc:
        for pgn, page in enumerate(doc):
            text = " ".join(page.get_text("text", clip=(0, 0, 400, 60)).split("\n"))
            text = re.sub("[^A-Za-z0-9\\s\\-\\(\\).,]+|\u2028", "", text).strip()
            print(text)
            if matches := re.findall(pattern, text, re.DOTALL):
                title[pgn] = " ".join([_ for _ in matches[0].strip().split(" ") if _])
                print(pgn,matches[0])
    return title
title = get_proper_fund_names(sample_path)

In [ ]:
import os ,json 
import pandas as pd

full_name = set()
split_name = set()
MANAGER_REGEX = FundRegex().MANAGER_STOP_WORDS
for a,b,files in os.walk(r"C:\Users\Kaustubh.keny\Projects\office-work\mywork-repo\sql_learn\json\MAR25DATA"):
    for paths in files:
        sample_path = os.path.join(os.getcwd(),"..","sql_learn","json","MAR25DATA",paths)
        # print(sample_path)
        try:
            with open(sample_path, "r", encoding="utf-8") as file:
                data = json.load(file)
                for k,scheme in data.items():
                    # print(k)
                    if "fund_manager" in scheme:
                        for entry in scheme['fund_manager']:
                            # print(entry['name'])
                            name = entry['name']
                            for regex_ in MANAGER_REGEX:
                                name = re.sub(f"\\b{regex_}\\b|[^A-Za-z\\s0-9]+","",name, re.IGNORECASE)
                                name = re.sub(r"\s+"," ",name)
                            full_name.add(name)
                            printthis = name if name.strip() else "EMPTY"
                            # print(f"<<{printthis}>>")
                            if printthis == "EMPTY":
                                print(k,printthis,entry['name'])
                            # split_name.add(name.split(' '))
                    # print(scheme.keys())
        except Exception as e:
            print(f"NEVER MIND {e}")
    # print(files)
        

In [ ]:
def _normalize_alphanumeric(text: str) -> str:
    if not isinstance(text,str):
        return text
    text = re.sub(r"[^a-zA-Z0-9]+", " ", str(text))
    return re.sub(r"\s+", " ", text).strip().lower()

def extract_clipped_data(input:str, pages:list, bboxes:list):
        
        document = fitz.open(input)
        final_list = []
    
        for pgn in pages:
            page = document[pgn]
            
            all_blocks = [] #store every data from bboxes
            
            for bbox in bboxes:
                blocks, seen_blocks = [], set()  #store unique blocks based on content and bbox
                
                page_blocks = page.get_text('dict', clip=bbox)['blocks']
                for block in page_blocks:
                    if block['type'] == 0 and 'lines' in block: #type 0 means text block
                        #hash_key
                        block_key = (tuple(block['bbox']), tuple(tuple(line['spans'][0]['text'] for line in block['lines'])))
                        if block_key not in seen_blocks:
                            seen_blocks.add(block_key)
                            blocks.append(block)

                sorted_blocks = sorted(blocks, key=lambda x: (x['bbox'][1], x['bbox'][0]))
                all_blocks.append(sorted_blocks)

            final_list.append({
                "pgn": pgn,
                "block": all_blocks #will be list[list,list,..]
            })

        document.close()
        return final_list
    
def extract_data_relative_line(path: str, line_x: float, side: str):
    doc = fitz.open(path)
    pages = doc.page_count

    final_list = []

    for pgn in range(pages):
        page = doc[pgn]

        blocks = page.get_text("dict")["blocks"]
        sorted_blocks = sorted(blocks, key=lambda x: (x["bbox"][1], x["bbox"][0]))
        extracted_blocks = []

        # Keep track of blocks to avoid duplicates
        added_blocks = set()

        for block in sorted_blocks:
            block_id = id(block)  # Unique identifier for the block

            for line in block.get("lines", []):
                for span in line.get("spans", []):
                    origin = span["origin"]
                    x0, _ = origin

                    # Check the side condition
                    if side == "left" and x0 < line_x and block_id not in added_blocks:
                        extracted_blocks.append(block)
                        added_blocks.add(block_id)  # Mark block as added
                    elif side == "right" and x0 > line_x and block_id not in added_blocks:
                        extracted_blocks.append(block)
                        added_blocks.add(block_id)  # Mark block as added

      
        final_list.append({
            "pgn": pgn,
            "blocks": extracted_blocks
        })

    doc.close()

    return final_list
  
def get_clipped_data(input:str, bboxes:list[set], *args):
    
        document = fitz.open(input)
        final_list = []
        if args:
            pages = list(args)
        else:
            pages = [i for i in document.page_count]
        
        for pgn in pages:
            page = document[pgn]

            blocks = []
            for bbox in bboxes:
                blocks.extend(page.get_text('dict', clip = bbox)['blocks']) #get all blocks
            
            filtered_blocks = [block for block in blocks if block['type']== 0 and 'lines' in block]
            # sorted_blocks = sorted(filtered_blocks, key= lambda x: (x['bbox'][1], x['bbox'][0]))
            
             # Extract text from sorted blocks
            extracted_text = []
            for block in filtered_blocks:
                block_text = []
                for line in block['lines']:
                    line_text = " ".join(span['text'] for span in line['spans'])
                    block_text.append(line_text)
                extracted_text.append("\n".join(block_text))
            
            final_list.append({
            "pgn": pgn,
            "block": filtered_blocks,
            "text": extracted_text
            })
            
            
        document.close()
        return final_list
    
def extract_clipped_text_all_pages(pdf_path, clip_coords):
    results = {}
    doc = fitz.open(pdf_path)
    clip_rect = fitz.Rect(*clip_coords)
    try:
        for page_number, page in enumerate(doc):
            text = page.get_text("text", clip=clip_rect).strip()
            results[page_number] = text
    finally:
        doc.close()
    return results